In [1]:
import pickle
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk
import pandas as pd
import spacy
import time

# NOTE: stopwords include pronouns! TODO: make custom stop words list?
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/srutikanthan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/srutikanthan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/srutikanthan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize


stop_words = set(stopwords.words("english"))
keep_stop = set(['she', 'her', 'hers', 'herself'])
mod_stop_words = stop_words - keep_stop

In [3]:
lmtzr = WordNetLemmatizer()

In [71]:

import string
import unicodedata
import re


def comments_clean(x):
    # define punctuation
    punctuations = '''"--"!()[]{};:'=+"\,<>./?@#$%^&*_~%'''

    # remove all punctuation from comments
    x['no_punct_col'] = x['comment'].copy() #The values in no_punct_col will be overwritten at the end of the for row loop
    i = 0
    for row in x['comment']: # looking at each row
        no_punct_str = ""
        for char in row:
            if char not in punctuations:
                no_punct_str = no_punct_str + char
        x['no_punct_col'][i] = no_punct_str # overwrite value in no_punct_col
        i += 1
    
    # lowercase, strip trailing whitespace, remove numbers, normalize text
    x['cleaned_text'] = x['no_punct_col'].str.lower() 
    x['cleaned_text'] = x['cleaned_text'].str.strip()
    x['cleaned_text'] = x['cleaned_text'].str.strip('\n')
    x['cleaned_text'] = x['cleaned_text'].str.strip('\n\n') # get rid of double returns/enters
    x['cleaned_text'] = x['cleaned_text'].str.replace(r'--', ' ')
    x['cleaned_text'] = x['cleaned_text'].str.replace(r'\.http.*', '')  
    x['no_punct_text'] = x['cleaned_text'].str.replace('\d+', '') # remove numbers
    x['no_punct_text'] = x['no_punct_text'].str.strip() # remove leading and trailing whitespace/escape chars
    x['no_punct_text'] = x['no_punct_text'].apply(lambda text: unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii'))
    
    cleaned_comments = x['no_punct_text'].to_list()
    # lemmatized comments
    lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in cleaned_comments]
    # stopwords and hyperlinks removed
    no_stopwords = [[word for word in l if word not in mod_stop_words and word != 'wa' and word != "" and word not in punctuations and "'" not in word and '`' not in word and '.' not in word]
              for l in lemmatized]
    no_stopwords = [[word for word in l if 'http' not in word]
              for l in no_stopwords]
    
    return no_stopwords
        
    
        
            

In [85]:
comments_corpus = pickle.load(open("comments_corpus_RC_2016-01.pickle","rb"))

# change corpus to dataframe
subreddit_lst = []
comment_lst = []
time_lst = []
for key in comments_corpus:
    for comment, time in comments_corpus[key]:
        subreddit_lst.append(key)
        comment_lst.append(comment)
        time_lst.append(time)
        
comments_df = pd.DataFrame(list(zip(subreddit_lst, comment_lst, time_lst)),
                           columns = ['subreddit', 'comment', 'time created'])

In [86]:
lemm = comments_clean(comments_df)
lemm

<ipython-input-71-d0f2a3916d12>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['no_punct_col'][i] = no_punct_str # overwrite value in no_punct_col


[['fuck', 'would', 'friend', 'ex'],
 ['great',
  'story',
  'full',
  'rp',
  'truth',
  'thanks',
  'consider',
  'making',
  'post',
  'trp',
  'front',
  'page',
  'someday'],
 ['let',
  'sus',
  'one',
  'shes',
  'either',
  'got',
  'high',
  'paying',
  'job',
  'clueless',
  'beta',
  'doormat',
  'her',
  'pocket',
  'either',
  'way',
  'resource',
  'box',
  'checked',
  'mean',
  'male',
  'she',
  'need',
  'big',
  'strong',
  'bull',
  'man',
  'wont',
  'find',
  'silicon',
  'valley',
  'effect',
  'become',
  'pronounced',
  'time',
  'typical',
  'afbb',
  'system',
  'work',
  'men',
  'category',
  'accessible',
  'woman',
  'soon',
  'woman',
  'begin',
  'outearning',
  'men',
  'en',
  'masse',
  'society',
  'begin',
  'winnow',
  'sign',
  'masculine',
  'behavior',
  'male',
  'kid',
  'teenages',
  'mean',
  'woman',
  'ruthless',
  'hunt',
  'alpha',
  'resouce',
  'need',
  'addressed',
  'totally',
  'ignore',
  'man',
  'even',
  'moderate',
  'beta',
  

## Using lemmatized words for other model analyses, using `no_punct_text` for sentiment analysis

In [30]:
len(lemm)

22555

In [35]:
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [87]:
comments_df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in comments_df['no_punct_text']]

In [88]:
comments_df

,subreddit,comment,time created,no_punct_col,cleaned_text,no_punct_text,neg
0,TheRedPill,Why the fuck would you be friends with an ex?,1451606442,Why the fuck would you be friends with an ex,why the fuck would you be friends with an ex,why the fuck would you be friends with an ex,0.240
1,TheRedPill,Great story full of RP truths. Thanks. Conside...,1451606479,Great story full of RP truths Thanks Consider ...,great story full of rp truths thanks consider ...,great story full of rp truths thanks consider ...,0.000
2,TheRedPill,Lets suss this out. \n\nFor one; she's either ...,1451606497,Lets suss this out \n\nFor one shes either got...,lets suss this out \n\nfor one shes either got...,lets suss this out \n\nfor one shes either got...,0.046
3,TheRedPill,Sounds like you may need some more treatment f...,1451606532,Sounds like you may need some more treatment f...,sounds like you may need some more treatment f...,sounds like you may need some more treatment f...,0.025
4,TheRedPill,"hack her phone or fb, its the only way you'll ...",1451606539,hack her phone or fb its the only way youll tr...,hack her phone or fb its the only way youll tr...,hack her phone or fb its the only way youll tr...,0.000
...,...,...,...,...,...,...,...
51941,technews,I'm consistently impressed with how quickly Am...,1454283541,Im consistently impressed with how quickly Ama...,im consistently impressed with how quickly ama...,im consistently impressed with how quickly ama...,0.000
51942,technews,I personally doubt Twitter is going anywhere.....,1454283616,I personally doubt Twitter is going anywhere t...,i personally doubt twitter is going anywhere t...,i personally doubt twitter is going anywhere t...,0.132
51943,technews,I still get peeved that we outsource animation...,1454283966,I still get peeved that we outsource animation...,i still get peeved that we outsource animation...,i still get peeved that we outsource animation...,0.054
51944,technews,Color me shocked.,1454284024,Color me shocked,color me shocked,color me shocked,0.535


------------------------------------------------------------------------------------------------------------
`compound` is the most useful gauge of sentiment for an entire sentence: it's a ‘normalized, weighted composite score computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive).


In [89]:
comments_df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in comments_df['no_punct_text']]

In [90]:
comments_df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in comments_df['no_punct_text']]

In [91]:
comments_df

,subreddit,comment,time created,no_punct_col,cleaned_text,no_punct_text,neg,compound,pos
0,TheRedPill,Why the fuck would you be friends with an ex?,1451606442,Why the fuck would you be friends with an ex,why the fuck would you be friends with an ex,why the fuck would you be friends with an ex,0.240,-0.1027,0.212
1,TheRedPill,Great story full of RP truths. Thanks. Conside...,1451606479,Great story full of RP truths Thanks Consider ...,great story full of rp truths thanks consider ...,great story full of rp truths thanks consider ...,0.000,0.8689,0.412
2,TheRedPill,Lets suss this out. \n\nFor one; she's either ...,1451606497,Lets suss this out \n\nFor one shes either got...,lets suss this out \n\nfor one shes either got...,lets suss this out \n\nfor one shes either got...,0.046,0.1796,0.044
3,TheRedPill,Sounds like you may need some more treatment f...,1451606532,Sounds like you may need some more treatment f...,sounds like you may need some more treatment f...,sounds like you may need some more treatment f...,0.025,0.8020,0.132
4,TheRedPill,"hack her phone or fb, its the only way you'll ...",1451606539,hack her phone or fb its the only way youll tr...,hack her phone or fb its the only way youll tr...,hack her phone or fb its the only way youll tr...,0.000,0.6478,0.237
...,...,...,...,...,...,...,...,...,...
51941,technews,I'm consistently impressed with how quickly Am...,1454283541,Im consistently impressed with how quickly Ama...,im consistently impressed with how quickly ama...,im consistently impressed with how quickly ama...,0.000,0.5859,0.304
51942,technews,I personally doubt Twitter is going anywhere.....,1454283616,I personally doubt Twitter is going anywhere t...,i personally doubt twitter is going anywhere t...,i personally doubt twitter is going anywhere t...,0.132,-0.5719,0.000
51943,technews,I still get peeved that we outsource animation...,1454283966,I still get peeved that we outsource animation...,i still get peeved that we outsource animation...,i still get peeved that we outsource animation...,0.054,0.0516,0.057
51944,technews,Color me shocked.,1454284024,Color me shocked,color me shocked,color me shocked,0.535,-0.3182,0.000


In [92]:
neg_comments = comments_df.loc[(comments_df['compound']<0)]
neg_comments

,subreddit,comment,time created,no_punct_col,cleaned_text,no_punct_text,neg,compound,pos
0,TheRedPill,Why the fuck would you be friends with an ex?,1451606442,Why the fuck would you be friends with an ex,why the fuck would you be friends with an ex,why the fuck would you be friends with an ex,0.240,-0.1027,0.212
6,TheRedPill,"Brazilian here too. Man, dont give up. I would...",1451606639,Brazilian here too Man dont give up I would re...,brazilian here too man dont give up i would re...,brazilian here too man dont give up i would re...,0.020,-0.0772,0.000
7,TheRedPill,&gt;She thought she could revist the days of g...,1451606651,gtShe thought she could revist the days of get...,gtshe thought she could revist the days of get...,gtshe thought she could revist the days of get...,0.096,-0.5423,0.000
8,TheRedPill,Took me 40 years. Keep exploring and keep read...,1451606653,Took me 40 years Keep exploring and keep readi...,took me 40 years keep exploring and keep readi...,took me years keep exploring and keep reading...,0.099,-0.5719,0.000
12,TheRedPill,none of that shit applies to foreigners and yo...,1451606743,none of that shit applies to foreigners and yo...,none of that shit applies to foreigners and yo...,none of that shit applies to foreigners and yo...,0.209,-0.7231,0.124
...,...,...,...,...,...,...,...,...,...
51935,technews,Bit different there. Everything you mentioned ...,1454164716,Bit different there Everything you mentioned r...,bit different there everything you mentioned r...,bit different there everything you mentioned r...,0.047,-0.2565,0.047
51938,technews,"Except Chromium did some sketchy shit as well,...",1454209237,Except Chromium did some sketchy shit as well ...,except chromium did some sketchy shit as well ...,except chromium did some sketchy shit as well ...,0.170,-0.5853,0.119
51942,technews,I personally doubt Twitter is going anywhere.....,1454283616,I personally doubt Twitter is going anywhere t...,i personally doubt twitter is going anywhere t...,i personally doubt twitter is going anywhere t...,0.132,-0.5719,0.000
51944,technews,Color me shocked.,1454284024,Color me shocked,color me shocked,color me shocked,0.535,-0.3182,0.000


In [93]:
comments_df.to_csv('sent_scores_comments.csv')
neg_comments.to_csv('neg_score_comments.csv')

In [94]:
all_neg = comments_clean(neg_comments)
all_neg

<ipython-input-71-d0f2a3916d12>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['no_punct_col'] = x['comment'].copy() #The values in no_punct_col will be overwritten at the end of the for row loop
<ipython-input-71-d0f2a3916d12>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['no_punct_col'][i] = no_punct_str # overwrite value in no_punct_col
<ipython-input-94-ff49d806d44f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

[['fuck', 'would', 'friend', 'ex'],
 ['would', 'counter', 'shit', 'test', 'entertaining', 'manner'],
 ['please',
  'dont',
  'encourage',
  'green',
  'text',
  'bullshit',
  'men',
  'write',
  'thought',
  'paragraph'],
 ['one',
  'major',
  'point',
  'she',
  'fucking',
  'cheated',
  'doe',
  'know',
  'mirror',
  'broken',
  'never',
  'put',
  'back',
  'together',
  'proper',
  'reflection',
  'fuck',
  'leading',
  'kitchen',
  'guy',
  'need',
  'crash',
  'course',
  'nexting'],
 ['gtthis',
  'loser',
  'behind',
  'bar',
  'rude',
  'would',
  'told',
  'her',
  'learn',
  'manner',
  'especially',
  'plenty',
  'choice',
  'gt',
  'girl',
  'approached',
  'throughout',
  'night'],
 ['thought',
  'military',
  'age',
  'fitness',
  'requirement',
  'ugly',
  'doe',
  'slim',
  'young',
  'athletic',
  'woman',
  'consider',
  'her',
  'doe',
  'gender',
  'ratio',
  'attract',
  'ugly',
  'woman'],
 ['see',
  'another',
  'angle',
  'think',
  'she',
  'know',
  'shes',
  

In [95]:
flat_neg = sum(all_neg, [])


In [96]:
flat_neg

['fuck',
 'would',
 'friend',
 'ex',
 'would',
 'counter',
 'shit',
 'test',
 'entertaining',
 'manner',
 'please',
 'dont',
 'encourage',
 'green',
 'text',
 'bullshit',
 'men',
 'write',
 'thought',
 'paragraph',
 'one',
 'major',
 'point',
 'she',
 'fucking',
 'cheated',
 'doe',
 'know',
 'mirror',
 'broken',
 'never',
 'put',
 'back',
 'together',
 'proper',
 'reflection',
 'fuck',
 'leading',
 'kitchen',
 'guy',
 'need',
 'crash',
 'course',
 'nexting',
 'gtthis',
 'loser',
 'behind',
 'bar',
 'rude',
 'would',
 'told',
 'her',
 'learn',
 'manner',
 'especially',
 'plenty',
 'choice',
 'gt',
 'girl',
 'approached',
 'throughout',
 'night',
 'thought',
 'military',
 'age',
 'fitness',
 'requirement',
 'ugly',
 'doe',
 'slim',
 'young',
 'athletic',
 'woman',
 'consider',
 'her',
 'doe',
 'gender',
 'ratio',
 'attract',
 'ugly',
 'woman',
 'see',
 'another',
 'angle',
 'think',
 'she',
 'know',
 'shes',
 'hot',
 'enough',
 'kept',
 'rock',
 'star',
 'she',
 'get',
 'wax',
 'wistful'

In [97]:
def count_tokens(tokens):
    '''
    Counts each distinct token (entity) in a list of tokens
    Inputs:
        tokens: list of tokens (must be immutable)
    Returns: dictionary that maps tokens to counts
    '''
    count_freq = {}

    for el in tokens:
    # Check if element is already in the dict
        if el in count_freq:
        # If it already is, add 1 to count
            count_freq[el] += 1
        else:
        # If it isn't, add one; also initializes key
            count_freq[el] = 1

    return count_freq

In [98]:
freq_neg_words = count_tokens(flat_neg)
sorted_freq_neg = dict( sorted(freq_neg_words.items(),
                           key=lambda item: item[1],
                           reverse=True))
sorted_freq_neg

{'woman': 7162,
 'she': 6501,
 'her': 6380,
 'get': 4710,
 'like': 4271,
 'people': 3801,
 'would': 3698,
 'one': 3401,
 'men': 3261,
 'shit': 2860,
 'think': 2820,
 'guy': 2803,
 'even': 2742,
 'fuck': 2738,
 'ha': 2651,
 'time': 2647,
 'want': 2646,
 'girl': 2528,
 'know': 2511,
 'thing': 2487,
 'make': 2370,
 'gt': 2342,
 'way': 2163,
 'go': 2025,
 'man': 1963,
 'doe': 1920,
 'say': 1870,
 'life': 1768,
 'need': 1721,
 'year': 1685,
 'right': 1666,
 'really': 1649,
 'dont': 1635,
 'going': 1623,
 'see': 1562,
 'much': 1531,
 'also': 1477,
 'never': 1461,
 'could': 1452,
 'good': 1445,
 'point': 1434,
 'fucking': 1428,
 'something': 1391,
 'take': 1384,
 'work': 1357,
 'sex': 1329,
 'still': 1320,
 'got': 1313,
 'mean': 1199,
 'look': 1194,
 'problem': 1176,
 'getting': 1140,
 'said': 1137,
 'beta': 1121,
 'lot': 1113,
 'bad': 1113,
 'come': 1108,
 'many': 1078,
 'give': 1076,
 'someone': 1076,
 'feel': 1068,
 'well': 1062,
 'back': 1058,
 'im': 1056,
 'actually': 1052,
 'rape': 1040